# inference 
---  
xgboost 0902

In [2]:
import xgboost as xgb 
import pandas as pd 
import numpy as np 

In [5]:
filename = "./models/xgb_0902.json"
model = xgb.XGBRegressor()
model.load_model(filename)

model

XGBRegressor(alpha=0.0001374641651407281, base_score=0.5, booster='gbtree',
             colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.9860960465630069, eta=0.07998970719169957,
             gamma=3.4625586401096294e-06, gpu_id=-1, importance_type='gain',
             interaction_constraints='', lambda=0.0024966053245291764,
             learning_rate=0.300000012, max_delta_step=0, max_depth=4,
             min_child_weight=1, min_sample_weight=2.2596596608740787,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=2,
             num_parallel_tree=1, objective='reg:squarederror', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=0.8776840962828512, tree_method='auto',
             validate_parameters=1, ...)

In [57]:
test = pd.read_csv("./data/processed/test.csv")
submit = pd.read_csv("./data/raw/sample.csv")

In [58]:
test["moving_avg"] = 0 
test["sale_before1"] = 0 

train = pd.read_csv("./data/processed/train.csv")
train_final_sale = train.loc[206, "y"]
window = train.loc[train.quarter == 3, "moving_avg"].mean()

test["moving_avg"] = window 
test.loc[0, "sale_before1"] = 24
test.head()

,week,soldout,kcal,event,payday,weather,precipitation,temperature,holiday,kcal_isna,...,weather_before1,payday_before1,payday_after1,year,month,quarter,day,temperature_avg,moving_avg,sale_before1
0,2,1,420.0,0,0,2,0.0,20.2,0,0,...,1,0,0,2014,79.05067,4,0.000000,1.589657,61.172024,24
1,1,0,415.0,0,0,1,0.0,23.9,0,0,...,0,0,0,2014,79.05067,4,0.033333,1.880832,61.172024,0
2,4,0,405.0,0,0,0,0.0,28.7,2,0,...,2,0,0,2014,79.05067,4,0.066667,2.258572,61.172024,0
3,0,1,400.0,0,0,2,0.5,21.5,1,0,...,0,0,0,2014,79.05067,4,0.166667,1.691962,61.172024,0
4,3,0,430.0,0,0,0,0.0,22.1,0,0,...,0,0,0,2014,79.05067,4,0.200000,1.739179,61.172024,0


In [59]:
def inference(test, model):
    prediction = []
    for i in range(test.shape[0]):
        data = test.loc[i, :].values.reshape(1, -1)
        predict = model.predict(data)
        predict = int(predict[0])
        test.loc[i+1, "sale_before1"] = predict
        prediction.append(predict)
    return prediction 
        
        
predict = inference(test, model)

In [61]:
submit = pd.read_csv("./data/raw/test.csv")
submit = submit[["datetime"]]
submit["y"] = predict 

In [63]:
submit.to_csv("submit.csv", index=False, header=None)

## result: 11.785